In [0]:
#шаг1: скачать репозиторий FactFuEval-2016 (https://github.com/dialogue-evaluation/factRuEval-2016)
#шаг2: внутри репозитория создаем директории FactRuEval2016_results и FactRuEval2016_results/results_of_bert_and_crf (для результатов)
#шаг3: устанавливаем библиотеку deep_ner (pip install deep_ner)
#шаг4: для обучения на наших текстах будем использовать предобученную BERT модель (https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1)

#!unzip factRuEval-2016-master.zip
#!pip install deep_ner

#требования по библиотекам:
#nltk==3.4.5
#numpy==1.18.1
#scikit-learn==0.22.1
#scipy==1.4.1
#tensorboard==2.1.0
#tensorflow==1.15.0
#tensorflow-hub==0.8.0
#bert-tensorflow==1.0.1
#spacy-udpipe==0.2.0
#spacy==2.2.3
#pymorphy2==0.8
#rusenttokenize==0.0.5

#работа выполена в google.colab
#python 3.6.9

In [0]:
from deep_ner.bert_ner import BERT_NER
from deep_ner.utils import load_dataset_from_json, factrueval2016_to_json
import json
import os
import sys

#sys.version
#sys.path.append('/content/factRuEval-2016-master/scripts')
#sys.path.append('/content/factRuEval-2016-master')
from scripts.dialent.task1.eval import Evaluator

In [12]:
%%time
factrueval2016_to_json("factRuEval-2016-master/devset", "factrueval2016devset_to_json.json")
X, y = load_dataset_from_json("factrueval2016devset_to_json.json")

bert_will_be_tuned = False

recognizer = BERT_NER(
    
            finetune_bert=bert_will_be_tuned, 
            batch_size=16, 
            l2_reg=1e-2,
            bert_hub_module_handle='https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1', 
            lstm_units=None, 
            validation_fraction=0.25,
            max_epochs=2, 
            patience=10, 
            gpu_memory_frac=0.9, 
            verbose=True, 
            random_seed=42,
            lr=1e-5 if bert_will_be_tuned else 1e-3, 
            udpipe_lang='ru', 
            use_additional_features=False
        )

recognizer.fit(X, y)

Downloaded pre-trained UDPipe model for 'ru' language
CPU times: user 1h 39min 40s, sys: 1min 16s, total: 1h 40min 56s
Wall time: 52min 28s


In [0]:
#!mkdir FactRuEval2016_results
#!mkdir FactRuEval2016_results/results_of_bert_and_crf

In [14]:
%%time
factrueval2016_to_json("factRuEval-2016-master/testset", "factrueval2016testset_to_json.json")

with open("factrueval2016testset_to_json.json", 'r') as fp:
  data_for_testing = json.load(fp)

_, true_entities = load_dataset_from_json("factrueval2016testset_to_json.json")

texts = []
additional_info = []
for cur_document in data_for_testing:
  base_name = os.path.join("FactRuEval2016_results/results_of_bert_and_crf", cur_document['base_name'] + '.task1')
  for cur_paragraph in cur_document['paragraph_bounds']:
    texts.append(cur_document['text'][cur_paragraph[0]:cur_paragraph[1]])
    additional_info.append((base_name, cur_paragraph))

predicted_entities = recognizer.predict(texts)

results_for_factrueval_2016 = dict()
for sample_idx, cur_result in enumerate(predicted_entities):
    base_name, paragraph_bounds = additional_info[sample_idx]
    for entity_type in cur_result:
        if entity_type == 'ORG':
            prepared_entity_type = 'org'
        elif entity_type == 'PERSON':
            prepared_entity_type = 'per'
        elif entity_type == 'LOCATION':
            prepared_entity_type = 'loc'
        else:
            prepared_entity_type = None
        
        for entity_bounds in cur_result[entity_type]:
            postprocessed_entity = (
                prepared_entity_type,
                entity_bounds[0] + paragraph_bounds[0],
                entity_bounds[1] - entity_bounds[0]
                )
            if base_name in results_for_factrueval_2016:
                results_for_factrueval_2016[base_name].append(postprocessed_entity)
            else:
                results_for_factrueval_2016[base_name] = [postprocessed_entity]

for base_name in results_for_factrueval_2016:
    with open(base_name, 'w+') as fp:
        for cur_entity in sorted(results_for_factrueval_2016[base_name], key=lambda it: (it[1], it[2], it[0])):
            fp.write('{0} {1} {2}\n'.format(cur_entity[0], cur_entity[1], cur_entity[2]))

CPU times: user 1h 3min 25s, sys: 40.1 s, total: 1h 4min 5s
Wall time: 33min 26s


In [15]:
#!python factRuEval-2016-master/scripts/t1_eval.py -t /content/FactRuEval2016_results/results_of_bert_and_crf -s /content/factRuEval-2016-master/testset -l

is_locorg_allowed = False
e = Evaluator(is_locorg_allowed)
e.evaluate(std_path="/content/factRuEval-2016-master/testset", test_path="/content/FactRuEval2016_results/results_of_bert_and_crf")

Failed to load the standard of book_3954:
Unknown mention tag: Facility
Type    P        R        F1       TP1      TP2      In Std.  In Test.
per        0.6237   0.8809   0.7303  1179.50  1179.50     1339     1891
loc        0.6733   0.6755   0.6744   829.48   829.48     1228     1232
org        0.4124   0.4807   0.4439   756.68   756.68     1574     1835
overall    0.5578   0.6679   0.6079  2765.67  2765.67     4141     4958


{'loc': <dialent.common.metrics.Metrics at 0x7f9df8742e48>,
 'org': <dialent.common.metrics.Metrics at 0x7f9df873eda0>,
 'overall': <dialent.common.metrics.Metrics at 0x7f9df874c390>,
 'per': <dialent.common.metrics.Metrics at 0x7f9df87a4f98>}

In [0]:
#!zip -r /content/file.zip /content/FactRuEval2016_results